Imports

In [248]:
import pandas as pd

import requests
from bs4 import BeautifulSoup

Load & Clean DataFrames

In [249]:
# Population by Districts - Wikipedia (Data Scraping)
url = "https://pt.wikipedia.org/wiki/Lista_de_distritos_portugueses_ordenados_por_popula%C3%A7%C3%A3o"

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:145.0) Gecko/20100101 Firefox/145.0"}

response = requests.get(url, headers = headers)
response

<Response [200]>

In [250]:
soup = BeautifulSoup(response.content, "html.parser")

allTables = soup.find_all("table", class_ = "wikitable")

#Selecting the second table on the page
allTables[1]

df_population = pd.read_html(str(allTables[1]))[0]

#Renaming columns
df_population = df_population.rename(columns = {'Distrito':'district'})

#Dropping unnecessary columns
df_population = df_population.drop(['2011', '2012', '2013', '2014'], axis=1)

#Removing unnecessary rows
df_population = df_population[df_population['district'] != 'Portugal']
df_population = df_population[df_population['district'] != 'Madeira']
df_population = df_population[df_population['district'] != 'Açores']

#Resetting index
df_population = df_population.reset_index(drop=True)

C:\Users\NocTo\AppData\Local\Temp\ipykernel_16884\3786360884.py:8: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_population = pd.read_html(str(allTables[1]))[0]


In [251]:
df_population

,district,2024,2023,2022,2021,2020,2019,2018,2017,2016,2015
0,Lisboa,2 390 715,2 355 867,2 321 239,2 287 869,2 286 240,2 292 204,2 272 417,2 264 699,2 255 301,2 249 708
1,Porto,1 860 255,1 846 178,1 825 192,1 803 223,1 792 631,1 794 995,1 789 353,1 787 706,1 787 443,1 789 527
2,Setúbal,916 859,902 863,891 652,882 588,880 218,870 130,862 886,860 213,857 719,856 479
3,Braga,867 537,863 547,856 089,850 128,849 547,844 782,839 750,837 525,836 634,836 797
4,Aveiro,734 762,725 461,714 615,708 902,703 994,701 611,698 564,698 351,698 949,700 812
5,Faro,492 747,484 122,478 618,469 983,469 892,465 610,459 597,456 113,453 300,451 679
6,Leiria,486 583,479 261,470 090,465 169,461 083,457 060,455 395,456 650,458 271,460 315
7,Santarém,446 393,441 255,435 500,431 633,427 502,424 719,425 389,428 358,432 015,435 867
8,Coimbra,423 432,418 136,412 878,411 510,410 830,411 305,411 125,412 970,415 273,417 741
9,Viseu,357 841,355 309,352 017,352 720,353 167,352 487,352 928,355 151,357 725,360 872


In [252]:
#Transforming the dataframe from wide to long format
df_population = df_population.melt(id_vars = ['district'], var_name = 'year', value_name = 'population')

#Sorting values by district, year and resetting index
df_population = df_population.sort_values(by=['district', 'year']).reset_index(drop=True)
df_population

,district,year,population
0,Aveiro,2015,700 812
1,Aveiro,2016,698 949
2,Aveiro,2017,698 351
3,Aveiro,2018,698 564
4,Aveiro,2019,701 611
...,...,...,...
175,Évora,2020,153 398
176,Évora,2021,153 507
177,Évora,2022,153 155
178,Évora,2023,153 475


In [253]:
df_population.dtypes

district      object
year          object
population    object
dtype: object

In [254]:
#Changing data types
df_population['year'] = df_population['year'].astype(int)
df_population['population'] = (df_population['population'].astype(str).str.replace(r'\s+', '', regex = True).astype(int))

In [255]:
df_population.dtypes

district      object
year           int64
population     int64
dtype: object

In [256]:
df_population

,district,year,population
0,Aveiro,2015,700812
1,Aveiro,2016,698949
2,Aveiro,2017,698351
3,Aveiro,2018,698564
4,Aveiro,2019,701611
...,...,...,...
175,Évora,2020,153398
176,Évora,2021,153507
177,Évora,2022,153155
178,Évora,2023,153475


In [257]:
#Average Gross Monthly Earnings per Employee - INE (CSV Files)
df_wage = pd.read_csv("https://raw.githubusercontent.com/antmgomes/FinalProject/refs/heads/main/Data/Average%20Monthly%20Gross%20Wage%20Per%20Worker%20-%20INE%20(FIXED).csv")
df_wage

,ANO,PAIS,PAIS2,SALARIO
0,2024,Portugal,PT,1604
1,2023,Portugal,PT,1507
2,2022,Portugal,PT,1412
3,2021,Portugal,PT,1362
4,2020,Portugal,PT,1315
5,2019,Portugal,PT,1277
6,2018,Portugal,PT,1241
7,2017,Portugal,PT,1216
8,2016,Portugal,PT,1196
9,2015,Portugal,PT,1179


In [258]:
#Renaming columns
df_wage = df_wage.rename(columns={'ANO':'year', 'SALARIO' : 'monthly_avg_gross_wage'})

#Dropping unnecessary columns
df_wage = df_wage.drop(['PAIS', 'PAIS2'], axis=1)

#Removing 2014 data
df_wage = df_wage[df_wage['year'] != 2014]

#Sorting values by year and resetting index
df_wage = df_wage.sort_values(by=['year']).reset_index(drop=True)
df_wage

,year,monthly_avg_gross_wage
0,2015,1179
1,2016,1196
2,2017,1216
3,2018,1241
4,2019,1277
5,2020,1315
6,2021,1362
7,2022,1412
8,2023,1507
9,2024,1604


In [259]:
#Housing Price by Districts - Idealista (CSV Files)
df_rental = pd.read_csv("https://raw.githubusercontent.com/antmgomes/FinalProject/refs/heads/main/Data/Rent%20Prices%20Report%202015-2024%20-%20Idealista%20(FIXED).csv")
df_sale = pd.read_csv("https://raw.githubusercontent.com/antmgomes/FinalProject/refs/heads/main/Data/Sales%20Prices%20Report%202015-2024%20-%20Idealista%20(FIXED).csv")

In [260]:
df_rental

,Districts,Price m²,Year,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Aveiro,"9,6 €/m2",2024,NaN,NaN,NaN,NaN
1,Beja,"10,2 €/m2",2024,NaN,NaN,NaN,NaN
2,Braga,"9,5 €/m2",2024,NaN,NaN,NaN,NaN
3,Castelo Branco,"7,3 €/m2",2024,NaN,NaN,NaN,NaN
4,Coimbra,"10,4 €/m2",2024,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
157,Santarém,"3,4 €/m2",2015,NaN,NaN,NaN,NaN
158,Setúbal,"4,8 €/m2",2015,NaN,NaN,NaN,NaN
159,Viana do Castelo,"4,2 €/m2",2015,NaN,NaN,NaN,NaN
160,Vila Real,"3,4 €/m2",2015,NaN,NaN,NaN,NaN


In [261]:
df_sale

,Districts,Price m²,Year,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,Aveiro,1.829 €/m²,2024,NaN,NaN,NaN,NaN
1,Beja,1.209 €/m²,2024,NaN,NaN,NaN,NaN
2,Braga,1.689 €/m²,2024,NaN,NaN,NaN,NaN
3,Bragança,917 €/m²,2024,NaN,NaN,NaN,NaN
4,Castelo Branco,908 €/m²,2024,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
239,Setúbal,968 €/m2,2015,NaN,NaN,NaN,NaN
240,Terceira (Ilha),956 €/m2,2015,NaN,NaN,NaN,NaN
241,Viana do Castelo,945 €/m2,2015,NaN,NaN,NaN,NaN
242,Vila Real,792 €/m2,2015,NaN,NaN,NaN,NaN


In [262]:
#Renaming columns
df_rental = df_rental.rename(columns={'Price m²':'price_rent_sqm', 'Districts':'district'})
df_sale = df_sale.rename(columns={'Price m²':'price_sale_sqm', 'Districts':'district'})

#Changing all column names to lowercase
df_rental.columns = df_rental.columns.str.lower()
df_sale.columns = df_sale.columns.str.lower()

#Keeping only relevant columns
df_rental = df_rental[['district', 'year', 'price_rent_sqm']]
df_sale = df_sale[['district', 'year', 'price_sale_sqm']]

#Removing '€/m²' from price columns
df_rental['price_rent_sqm'] = df_rental['price_rent_sqm'].str[ : -4]
df_sale['price_sale_sqm'] = df_sale['price_sale_sqm'].str[ : -4]

#Removing whitespace from price columns
df_rental['price_rent_sqm'] = df_rental['price_rent_sqm'].str.strip()
df_sale['price_sale_sqm'] = df_sale['price_sale_sqm'].str.strip()

In [263]:
df_rental

,district,year,price_rent_sqm
0,Aveiro,2024,"9,6"
1,Beja,2024,"10,2"
2,Braga,2024,"9,5"
3,Castelo Branco,2024,"7,3"
4,Coimbra,2024,"10,4"
...,...,...,...
157,Santarém,2015,"3,4"
158,Setúbal,2015,"4,8"
159,Viana do Castelo,2015,"4,2"
160,Vila Real,2015,"3,4"


In [264]:
df_sale

,district,year,price_sale_sqm
0,Aveiro,2024,1.829
1,Beja,2024,1.209
2,Braga,2024,1.689
3,Bragança,2024,917
4,Castelo Branco,2024,908
...,...,...,...
239,Setúbal,2015,968
240,Terceira (Ilha),2015,956
241,Viana do Castelo,2015,945
242,Vila Real,2015,792


In [265]:
df_rental.dtypes, df_sale.dtypes

(district          object
 year               int64
 price_rent_sqm    object
 dtype: object,
 district          object
 year               int64
 price_sale_sqm    object
 dtype: object)

In [266]:
#Changing data types
df_rental['price_rent_sqm'] = df_rental['price_rent_sqm'].str.replace(',', '.').astype(float)
df_sale['price_sale_sqm'] = df_sale['price_sale_sqm'].str.replace('.', '').astype(float)

df_rental.dtypes, df_sale.dtypes

(district           object
 year                int64
 price_rent_sqm    float64
 dtype: object,
 district           object
 year                int64
 price_sale_sqm    float64
 dtype: object)

In [267]:
df_rental['district'].unique()

array(['Aveiro', 'Beja', 'Braga', 'Castelo Branco', 'Coimbra', 'Évora',
       'Faro', 'Leiria', 'Lisboa', 'Madeira (Ilha)', 'Portalegre',
       'Porto', 'Santarém', 'São Miguel (ilha)', 'Setúbal',
       'Viana do Castelo', 'Vila Real', 'Viseu', 'Portugal', 'Guarda'],
      dtype=object)

In [268]:
df_sale['district'].unique()

array(['Aveiro', 'Beja', 'Braga', 'Bragança', 'Castelo Branco', 'Coimbra',
       'Évora', 'Faial (Ilha)', 'Faro', 'Guarda', 'Leiria', 'Lisboa',
       'Madeira (Ilha)', 'Pico (Ilha)', 'Portalegre', 'Porto',
       'Porto Santo (Ilha)', 'Santa Maria (Ilha)', 'Santarém',
       'São Jorge (Ilha)', 'São Miguel (ilha)', 'Setúbal',
       'Terceira (Ilha)', 'Viana do Castelo', 'Vila Real', 'Viseu',
       'Portugal'], dtype=object)

In [269]:
#Removing unnecessary rows
df_rental = df_rental[df_rental['district'] != 'Madeira (Ilha)']
df_rental = df_rental[df_rental['district'] != 'São Miguel (ilha)']
df_rental = df_rental[df_rental['district'] != 'Portugal']

df_sale = df_sale[df_sale['district'] != 'Faial (Ilha)']
df_sale = df_sale[df_sale['district'] != 'Madeira (Ilha)']
df_sale = df_sale[df_sale['district'] != 'Pico (Ilha)']
df_sale = df_sale[df_sale['district'] != 'Porto Santo (Ilha)']
df_sale = df_sale[df_sale['district'] != 'Santa Maria (Ilha)']
df_sale = df_sale[df_sale['district'] != 'São Jorge (Ilha)']
df_sale = df_sale[df_sale['district'] != 'São Miguel (ilha)']
df_sale = df_sale[df_sale['district'] != 'Açores']
df_sale = df_sale[df_sale['district'] != 'Terceira (Ilha)']
df_sale = df_sale[df_sale['district'] != 'Portugal']

In [270]:
df_rental['district'].unique()

array(['Aveiro', 'Beja', 'Braga', 'Castelo Branco', 'Coimbra', 'Évora',
       'Faro', 'Leiria', 'Lisboa', 'Portalegre', 'Porto', 'Santarém',
       'Setúbal', 'Viana do Castelo', 'Vila Real', 'Viseu', 'Guarda'],
      dtype=object)

In [271]:
df_sale['district'].unique()

array(['Aveiro', 'Beja', 'Braga', 'Bragança', 'Castelo Branco', 'Coimbra',
       'Évora', 'Faro', 'Guarda', 'Leiria', 'Lisboa', 'Portalegre',
       'Porto', 'Santarém', 'Setúbal', 'Viana do Castelo', 'Vila Real',
       'Viseu'], dtype=object)

In [272]:
#Sorting values by district, year and resetting index
df_rental = df_rental.sort_values(by = ['district', 'year']).reset_index(drop=True)
df_sale = df_sale.sort_values(by = ['district', 'year']).reset_index(drop=True)

In [273]:
df_rental

,district,year,price_rent_sqm
0,Aveiro,2015,3.9
1,Aveiro,2016,4.0
2,Aveiro,2017,4.4
3,Aveiro,2018,5.0
4,Aveiro,2019,6.3
...,...,...,...
140,Évora,2017,5.3
141,Évora,2020,7.1
142,Évora,2022,9.3
143,Évora,2023,10.3


In [274]:
df_sale

,district,year,price_sale_sqm
0,Aveiro,2015,819.0
1,Aveiro,2016,813.0
2,Aveiro,2017,880.0
3,Aveiro,2018,994.0
4,Aveiro,2019,1089.0
...,...,...,...
175,Évora,2020,871.0
176,Évora,2021,1044.0
177,Évora,2022,1272.0
178,Évora,2023,1322.0


In [275]:
#Merge df_rental and df_sale into df_idealista
df_idealista = pd.merge(df_rental, df_sale, on=['district', 'year'], how='outer')
df_idealista

,district,year,price_rent_sqm,price_sale_sqm
0,Aveiro,2015,3.9,819.0
1,Aveiro,2016,4.0,813.0
2,Aveiro,2017,4.4,880.0
3,Aveiro,2018,5.0,994.0
4,Aveiro,2019,6.3,1089.0
...,...,...,...,...
175,Évora,2020,7.1,871.0
176,Évora,2021,NaN,1044.0
177,Évora,2022,9.3,1272.0
178,Évora,2023,10.3,1322.0


In [276]:
df_idealista.isnull().sum()

district           0
year               0
price_rent_sqm    35
price_sale_sqm     0
dtype: int64

In [277]:
# Minium Wage and % of Population Receiving it - INE, Banco de Portugal (CSV Files)
df_min_wage = pd.read_csv("https://raw.githubusercontent.com/antmgomes/FinalProject/refs/heads/main/Data/Min%20Wage%20and%20%25%20of%20Pop%20Earning%20It%20-%20Banco%20de%20Portugal%20%26%20INE%20(FIXED).csv")
df_min_wage

,year,min_wage,percentage_pop_min_wage
0,2015,505,18.0
1,2016,530,21.9
2,2017,557,22.4
3,2018,580,22.0
4,2019,600,20.7
5,2020,635,23.4
6,2021,665,23.1
7,2022,705,22.8
8,2023,760,NaN
9,2024,820,NaN


In [278]:
df_min_wage.dtypes

year                         int64
min_wage                     int64
percentage_pop_min_wage    float64
dtype: object

In [279]:
#Tourist Accommodations - PORDATA (CSV File)
df_tourism = pd.read_csv("https://raw.githubusercontent.com/antmgomes/TheCostOfLivingInPortugal/refs/heads/main/Data/Tourist%20Accommodations%20-%20PORDATA.csv")
df_municipalities = pd.read_csv("https://raw.githubusercontent.com/antmgomes/TheCostOfLivingInPortugal/refs/heads/main/Data/Municipalities%20%26%20Districts%20of%20Portugal%20(FIXED).csv")

In [280]:
df_tourism

,01. Ano,02. Nome RegiÃ£o (Portugal),03. Ã‚mbito GeogrÃ¡fico,04. Indicador,05. Filtro 1,06. Filtro 2,07. Filtro 3,08. Escala,09. SÃ­mbolo,10. Valor
0,2017,Portugal,NaN,Alojamentos turÃ­sticos por tipologia adotada ...,1. Hotelaria,NaN,NaN,alojamentos,NaN,1758.0
1,2017,Portugal,NaN,Alojamentos turÃ­sticos por tipologia adotada ...,1.1. HotÃ©is,NaN,NaN,alojamentos,NaN,1309.0
2,2017,Portugal,NaN,Alojamentos turÃ­sticos por tipologia adotada ...,1.2. HotÃ©is-apartamentos,NaN,NaN,alojamentos,NaN,145.0
3,2017,Portugal,NaN,Alojamentos turÃ­sticos por tipologia adotada ...,1.3. Pousadas/Quintas da Madeira,NaN,NaN,alojamentos,NaN,46.0
4,2017,Portugal,NaN,Alojamentos turÃ­sticos por tipologia adotada ...,1.4. Apartamentos turÃ­sticos,NaN,NaN,alojamentos,NaN,203.0
...,...,...,...,...,...,...,...,...,...,...
64318,2016,Vouzela,MunicÃ­pio,Alojamentos turÃ­sticos por tipologia adotada ...,HotÃ©is-apartamentos,NaN,NaN,alojamentos,-,NaN
64319,2016,Vouzela,MunicÃ­pio,Alojamentos turÃ­sticos por tipologia adotada ...,MotÃ©is,NaN,NaN,alojamentos,-,NaN
64320,2016,Vouzela,MunicÃ­pio,Alojamentos turÃ­sticos por tipologia adotada ...,PensÃµes,NaN,NaN,alojamentos,NaN,6.0
64321,2016,Vouzela,MunicÃ­pio,Alojamentos turÃ­sticos por tipologia adotada ...,Pousadas,NaN,NaN,alojamentos,-,NaN


In [281]:
df_municipalities

,Municípios,Distritos
0,Abrantes,Santarém
1,Águeda,Aveiro
2,Aguiar da Beira,Guarda
3,Alandroal,Évora
4,Albergaria-a-Velha,Aveiro
...,...,...
303,Vimioso,Bragança
304,Vinhais,Bragança
305,Viseu,Viseu
306,Vizela,Braga


In [282]:
#Renaming columns
df_municipalities = df_municipalities.rename(columns={'Municípios':'municipality', 'Distritos':'district'})
df_tourism = df_tourism.rename(columns={'01. Ano':'year', '02. Nome RegiÃ£o (Portugal)':'municipality', '05. Filtro 1':'type', '10. Valor' : 'amount'})

#Remove unnecessary columns
df_tourism = df_tourism.drop(['06. Filtro 2', '07. Filtro 3', '09. SÃ­mbolo', '08. Escala', '03. Ã‚mbito GeogrÃ¡fico', '04. Indicador'], axis=1)

#Removing rows with year less than 2015
df_tourism = df_tourism[df_tourism['year'] >= 2015]

In [283]:
#Merging df_tourism with df_municipalities to get district column
df_tourismMerged = df_tourism.merge(df_municipalities, on='municipality', how='left')
df_tourismMerged

,year,municipality,type,amount,district
0,2017,Portugal,1. Hotelaria,1758.0,NaN
1,2017,Portugal,1.1. HotÃ©is,1309.0,NaN
2,2017,Portugal,1.2. HotÃ©is-apartamentos,145.0,NaN
3,2017,Portugal,1.3. Pousadas/Quintas da Madeira,46.0,NaN
4,2017,Portugal,1.4. Apartamentos turÃ­sticos,203.0,NaN
...,...,...,...,...,...
27805,2016,Vouzela,HotÃ©is-apartamentos,NaN,Viseu
27806,2016,Vouzela,MotÃ©is,NaN,Viseu
27807,2016,Vouzela,PensÃµes,6.0,Viseu
27808,2016,Vouzela,Pousadas,NaN,Viseu


In [284]:
df_tourismMerged['district'].unique()

array([nan, 'Santarém', 'Guarda', 'Évora', 'Aveiro', 'Faro', 'Setúbal',
       'Lisboa', 'Beja', 'Porto', 'Braga', 'Viana do Castelo', 'Coimbra',
       'Viseu', 'Portalegre', 'Leiria', 'Castelo Branco', 'Vila Real',
       'Açores', 'Madeira', 'Bragança'], dtype=object)

In [285]:
#Deleting Açores and Madeira entries
df_tourismMerged = df_tourismMerged[df_tourismMerged['district'] != 'Açores']
df_tourismMerged = df_tourismMerged[df_tourismMerged['district'] != 'Madeira']

#Deleting Nan entries from district column
df_tourismMerged = df_tourismMerged.dropna(subset=['district'])
df_tourismMerged

,year,municipality,type,amount,district
90,2017,Abrantes,1. Hotelaria,3.0,Santarém
91,2017,Abrantes,1.1. HotÃ©is,3.0,Santarém
92,2017,Abrantes,1.2. HotÃ©is-apartamentos,NaN,Santarém
93,2017,Abrantes,1.3. Pousadas/Quintas da Madeira,NaN,Santarém
94,2017,Abrantes,1.4. Apartamentos turÃ­sticos,NaN,Santarém
...,...,...,...,...,...
27805,2016,Vouzela,HotÃ©is-apartamentos,NaN,Viseu
27806,2016,Vouzela,MotÃ©is,NaN,Viseu
27807,2016,Vouzela,PensÃµes,6.0,Viseu
27808,2016,Vouzela,Pousadas,NaN,Viseu


In [286]:
df_tourismMerged['district'].unique()

array(['Santarém', 'Guarda', 'Évora', 'Aveiro', 'Faro', 'Setúbal',
       'Lisboa', 'Beja', 'Porto', 'Braga', 'Viana do Castelo', 'Coimbra',
       'Viseu', 'Portalegre', 'Leiria', 'Castelo Branco', 'Vila Real',
       'Bragança'], dtype=object)

In [287]:
df_tourismMerged['type'].unique()

array(['1. Hotelaria', '1.1. HotÃ©is', '1.2. HotÃ©is-apartamentos',
       '1.3. Pousadas/Quintas da Madeira',
       '1.4. Apartamentos turÃ\xadsticos',
       '1.5 Aldeamentos turÃ\xadsticos', '2. Alojamento local',
       '3. Turismo no espaÃ§o rural e de habitaÃ§Ã£o', 'Total',
       'Aldeamentos turÃ\xadsticos', 'Apartamentos turÃ\xadsticos',
       'Estalagens', 'HotÃ©is', 'HotÃ©is-apartamentos', 'MotÃ©is',
       'PensÃµes', 'Pousadas'], dtype=object)

In [288]:
#Deleting entries with 1.1. HotÃ©is in type column
df_tourismMerged = df_tourismMerged[df_tourismMerged['type'] != '1.1. HotÃ©is']
df_tourismMerged = df_tourismMerged[df_tourismMerged['type'] != '1.2. HotÃ©is-apartamentos']
df_tourismMerged = df_tourismMerged[df_tourismMerged['type'] != '1.3. Pousadas/Quintas da Madeira']
df_tourismMerged = df_tourismMerged[df_tourismMerged['type'] != '1.4. Apartamentos turÃ\xadsticos']
df_tourismMerged = df_tourismMerged[df_tourismMerged['type'] != '1.5 Aldeamentos turÃ\xadsticos']
df_tourismMerged = df_tourismMerged[df_tourismMerged['type'] != 'Total']

#Renaming type rows
df_tourismMerged['type'] = df_tourismMerged['type'].replace({'1. Hotelaria' : 'Hospitality', 
                                                           '2. Alojamento local' : 'Local Accommodation (AL)',
                                                           '3. Turismo no espaÃ§o rural e de habitaÃ§Ã£o' : 'Rural and Residential Tourism',

                                                           'Aldeamentos turÃ\xadsticos' : 'Hospitality',
                                                           'Apartamentos turÃ\xadsticos' : 'Hospitality',
                                                           'Estalagens' : 'Hospitality',
                                                           'HotÃ©is' : 'Hospitality',
                                                           'HotÃ©is-apartamentos' : 'Hospitality',
                                                           'MotÃ©is' : 'Hospitality',
                                                           'PensÃµes' : 'Hospitality',
                                                           'Pousadas' : 'Hospitality',})
df_tourismMerged

,year,municipality,type,amount,district
90,2017,Abrantes,Hospitality,3.0,Santarém
96,2017,Abrantes,Local Accommodation (AL),2.0,Santarém
97,2017,Abrantes,Rural and Residential Tourism,5.0,Santarém
108,2017,Aguiar da Beira,Hospitality,NaN,Guarda
114,2017,Aguiar da Beira,Local Accommodation (AL),1.0,Guarda
...,...,...,...,...,...
27804,2016,Vouzela,Hospitality,NaN,Viseu
27805,2016,Vouzela,Hospitality,NaN,Viseu
27806,2016,Vouzela,Hospitality,NaN,Viseu
27807,2016,Vouzela,Hospitality,6.0,Viseu


In [289]:
#Drop municipality column
df_tourismMerged = df_tourismMerged.drop(['municipality'], axis=1)
df_tourismMerged

#Reorder columns
df_tourismMerged = df_tourismMerged[['district', 'year', 'type', 'amount']]

#Sorting values by district, year and resetting index
df_tourismMerged = df_tourismMerged.sort_values(by=['district', 'year']).reset_index(drop=True)
df_tourismMerged

,district,year,type,amount
0,Aveiro,2015,Hospitality,0.0
1,Aveiro,2015,Hospitality,0.0
2,Aveiro,2015,Hospitality,0.0
3,Aveiro,2015,Hospitality,1.0
4,Aveiro,2015,Hospitality,0.0
...,...,...,...,...
8235,Évora,2024,Local Accommodation (AL),4.0
8236,Évora,2024,Rural and Residential Tourism,NaN
8237,Évora,2024,Hospitality,NaN
8238,Évora,2024,Local Accommodation (AL),NaN


In [290]:
#Turn all Nans in amount column to 0
df_tourismMerged["amount"] = pd.to_numeric(df_tourismMerged["amount"], errors="coerce").fillna(0)
df_tourismMerged

,district,year,type,amount
0,Aveiro,2015,Hospitality,0.0
1,Aveiro,2015,Hospitality,0.0
2,Aveiro,2015,Hospitality,0.0
3,Aveiro,2015,Hospitality,1.0
4,Aveiro,2015,Hospitality,0.0
...,...,...,...,...
8235,Évora,2024,Local Accommodation (AL),4.0
8236,Évora,2024,Rural and Residential Tourism,0.0
8237,Évora,2024,Hospitality,0.0
8238,Évora,2024,Local Accommodation (AL),0.0


In [291]:
#Aggregate by district, year and type (if rows have the same district, year and type then sum their amount values)
df_tourismMerged = (df_tourismMerged.groupby(["district", "year", "type"], as_index = False).agg({"amount": "sum"}))
df_tourismMerged

,district,year,type,amount
0,Aveiro,2015,Hospitality,118.0
1,Aveiro,2016,Hospitality,120.0
2,Aveiro,2017,Hospitality,48.0
3,Aveiro,2017,Local Accommodation (AL),63.0
4,Aveiro,2017,Rural and Residential Tourism,35.0
...,...,...,...,...
463,Évora,2023,Local Accommodation (AL),53.0
464,Évora,2023,Rural and Residential Tourism,92.0
465,Évora,2024,Hospitality,34.0
466,Évora,2024,Local Accommodation (AL),63.0


In [292]:
#Change to wide format without keeping type column
df_tourismMerged = df_tourismMerged.pivot_table(index=['district', 'year'], columns='type', values='amount').reset_index()

#Change index name to default
df_tourismMerged.columns.name = None
df_tourismMerged

,district,year,Hospitality,Local Accommodation (AL),Rural and Residential Tourism
0,Aveiro,2015,118.0,NaN,NaN
1,Aveiro,2016,120.0,NaN,NaN
2,Aveiro,2017,48.0,63.0,35.0
3,Aveiro,2018,54.0,64.0,37.0
4,Aveiro,2019,54.0,91.0,42.0
...,...,...,...,...,...
175,Évora,2020,28.0,38.0,74.0
176,Évora,2021,31.0,46.0,78.0
177,Évora,2022,34.0,47.0,78.0
178,Évora,2023,35.0,53.0,92.0


In [293]:
# Merging all dataframes
df_final = pd.merge(df_idealista, df_population, on = ['district', 'year'], how = 'outer')
df_final = pd.merge(df_final, df_wage, on = ['year'], how = 'outer')
df_final = pd.merge(df_final, df_min_wage, on = ['year'], how = 'outer')
df_final

,district,year,price_rent_sqm,price_sale_sqm,population,monthly_avg_gross_wage,min_wage,percentage_pop_min_wage
0,Aveiro,2015,3.9,819.0,700812,1179,505,18.0
1,Beja,2015,4.2,845.0,146748,1179,505,18.0
2,Braga,2015,3.5,745.0,836797,1179,505,18.0
3,Bragança,2015,NaN,662.0,128418,1179,505,18.0
4,Castelo Branco,2015,3.1,713.0,185203,1179,505,18.0
...,...,...,...,...,...,...,...,...
175,Setúbal,2024,13.4,2700.0,916859,1604,820,NaN
176,Viana do Castelo,2024,8.4,1481.0,234645,1604,820,NaN
177,Vila Real,2024,7.5,1056.0,184707,1604,820,NaN
178,Viseu,2024,7.3,1147.0,357841,1604,820,NaN


In [294]:
#Sort values by district, year and reset index
df_final = df_final.sort_values(by = ['district', 'year']).reset_index(drop=True)
df_final

,district,year,price_rent_sqm,price_sale_sqm,population,monthly_avg_gross_wage,min_wage,percentage_pop_min_wage
0,Aveiro,2015,3.9,819.0,700812,1179,505,18.0
1,Aveiro,2016,4.0,813.0,698949,1196,530,21.9
2,Aveiro,2017,4.4,880.0,698351,1216,557,22.4
3,Aveiro,2018,5.0,994.0,698564,1241,580,22.0
4,Aveiro,2019,6.3,1089.0,701611,1277,600,20.7
...,...,...,...,...,...,...,...,...
175,Évora,2020,7.1,871.0,153398,1315,635,23.4
176,Évora,2021,NaN,1044.0,153507,1362,665,23.1
177,Évora,2022,9.3,1272.0,153155,1412,705,22.8
178,Évora,2023,10.3,1322.0,153475,1507,760,NaN


In [295]:
df_final.isnull().sum()

district                    0
year                        0
price_rent_sqm             35
price_sale_sqm              0
population                  0
monthly_avg_gross_wage      0
min_wage                    0
percentage_pop_min_wage    36
dtype: int64

In [296]:
#Checking where my null values are located
condition = df_final[['price_rent_sqm', 'price_sale_sqm']].isna()
df_final[condition.any(axis=1)]

,district,year,price_rent_sqm,price_sale_sqm,population,monthly_avg_gross_wage,min_wage,percentage_pop_min_wage
12,Beja,2017,NaN,779.0,144758,1216,557,22.4
13,Beja,2018,NaN,793.0,144173,1241,580,22.0
14,Beja,2019,NaN,802.0,144859,1277,600,20.7
15,Beja,2020,NaN,777.0,145380,1315,635,23.4
16,Beja,2021,NaN,841.0,146353,1362,665,23.1
17,Beja,2022,NaN,934.0,147446,1412,705,22.8
30,Bragança,2015,NaN,662.0,128418,1179,505,18.0
31,Bragança,2016,NaN,701.0,127219,1196,530,21.9
32,Bragança,2017,NaN,686.0,126210,1216,557,22.4
33,Bragança,2018,NaN,688.0,125249,1241,580,22.0


In [297]:
#Final Merge with Tourism data (Added it later)
df_final = pd.merge(df_final, df_tourismMerged, on=['district', 'year'], how='outer')

#Sorting values by district, year and resetting index
df_final = df_final.sort_values(by = ['district', 'year']).reset_index(drop=True)
df_final

,district,year,price_rent_sqm,price_sale_sqm,population,monthly_avg_gross_wage,min_wage,percentage_pop_min_wage,Hospitality,Local Accommodation (AL),Rural and Residential Tourism
0,Aveiro,2015,3.9,819.0,700812,1179,505,18.0,118.0,NaN,NaN
1,Aveiro,2016,4.0,813.0,698949,1196,530,21.9,120.0,NaN,NaN
2,Aveiro,2017,4.4,880.0,698351,1216,557,22.4,48.0,63.0,35.0
3,Aveiro,2018,5.0,994.0,698564,1241,580,22.0,54.0,64.0,37.0
4,Aveiro,2019,6.3,1089.0,701611,1277,600,20.7,54.0,91.0,42.0
...,...,...,...,...,...,...,...,...,...,...,...
175,Évora,2020,7.1,871.0,153398,1315,635,23.4,28.0,38.0,74.0
176,Évora,2021,NaN,1044.0,153507,1362,665,23.1,31.0,46.0,78.0
177,Évora,2022,9.3,1272.0,153155,1412,705,22.8,34.0,47.0,78.0
178,Évora,2023,10.3,1322.0,153475,1507,760,NaN,35.0,53.0,92.0


Export CSV

In [298]:
df_final.to_csv("Final_Project_MERGED_Dataset.csv", index = False)